In [1]:
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info.operators import Operator
from qiskit.tools.monitor import job_monitor
import csv
from qiskit.providers.aer.noise.errors import thermal_relaxation_error
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer import StatevectorSimulator
from qiskit.circuit.library import MCMT
from qiskit.circuit.library import OR
from qiskit.transpiler.passes.basis import Unroller
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.providers.aer import AerSimulator
from tqdm import tqdm

def minus_iY():
    
    minus_iY = Operator(np.array([[0,-1],[1,0]]))
    target = QuantumRegister(1, 't_qubit')
    qc = QuantumCircuit(target)
    qc.unitary(minus_iY,[*target])
    
    return qc

def Ph(theta):
    ## Global phase transition gate
    Ph = Operator(np.e**(1j*theta)*np.array([[1,0],[0,1]]))
    target = QuantumRegister(1,'t_qubit')
    qc = QuantumCircuit(target)
    qc.unitary(Ph,[*target])
    return Ph

def computed_fidelity(theta,X_exp,Y_exp,Z_exp):
    computed_list = np.array([X_exp,Y_exp,Z_exp])
    expected_list = np.array([2*np.cos(theta)**2*np.sin(theta)**2, 
                              0, np.cos(theta)**4-np.sin(theta)**4])/p(theta)
    return 0.5*(1+np.dot(computed_list, expected_list))

def p(theta):
    return np.cos(theta)**4 + np.sin(theta)**4

def q(theta):
    return np.arctan((np.tan(theta))**2)

def new_fidelity(theta,x,y,z):
    return 0.5*(1+x*np.sin(2*q(theta))+z*np.cos(2*q(theta)))

In [2]:
def scm_experiment(theta, n, S, measure=None):
    
    '''
    S=0 for noisless, 
    S=1 for FakeMumbai
    S=2 for FakeMelbourne
    '''
    
    bath = QuantumRegister(n)
    reg_1 = QuantumRegister(1, 'reg_1')
    reg_2 = QuantumRegister(1, 'reg_2')
    target = QuantumRegister(1, 't_qubit')
    c = ClassicalRegister(1)
    circ = QuantumCircuit(bath, reg_2, reg_1, target, c)
    
    circ.x(reg_2) # Initialise R2=|1> for first iteration
    
    for i in range(1,n+1):
        ## U
        circ.cry(2*theta, reg_2, reg_1)
        circ.append(MCMT(minus_iY(),1,1),[reg_1,target])
        circ.cry(-2*theta, reg_2, reg_1)
        ## Reset
        circ.cry(np.pi/2, reg_1, target)
        ## R1&R2 interaction
        circ.cx(reg_2, reg_1)
        circ.cx(reg_1, reg_2)
        ## bath&R1 interaction
        circ.cx(reg_1, bath[i-1])
        circ.cx(bath[i-1], reg_1)
        
    #Pauli Measurements
    if measure == 'x':
        circ.h(target)
    elif measure == 'y':
        circ.sdg(target)
        circ.h(target)
    circ.measure(target,c)

    shots = 8192
    qcomp = Aer.get_backend('qasm_simulator')
    
    if S==0:
        job = execute(circ, backend = qcomp, shots = shots)
    elif S==1:
        from qiskit.providers.fake_provider import FakeMumbai
        backend = FakeMumbai()
        from qiskit.providers.aer.noise import NoiseModel
        noise_model = NoiseModel.from_backend(backend)
        coupling_map = backend.configuration().coupling_map
        basis_gates = noise_model.basis_gates
        job = execute(circ, 
                      backend = qcomp, 
                      shots = shots, 
                      coupling_map = coupling_map, 
                      basis_gates = basis_gates, 
                      noise_model = noise_model)
    elif S==2:
        from qiskit.providers.fake_provider import FakeMelbourne
        backend = FakeMelbourne()
        from qiskit.providers.aer.noise import NoiseModel
        noise_model = NoiseModel.from_backend(backend)
        coupling_map = backend.configuration().coupling_map
        basis_gates = noise_model.basis_gates
        job = execute(circ, 
                      backend = qcomp, 
                      shots = shots, 
                      coupling_map = coupling_map, 
                      basis_gates = basis_gates, 
                      noise_model = noise_model)
    result = job.result()
    result_dictionary = result.get_counts(circ)
    probs = {}
    for output in ['0','1']:
        if output in result_dictionary:
            probs[output] = result_dictionary[output]
        else:
            probs[output] = 0
    return (probs['0'] -  probs['1']) / shots

In [3]:
def acquiring_data():

    with open('SCM_combined.csv', 'w', newline='') as csvfile:

        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(['Theta','rounds','dat_clean','Xc','Yc','Zc',
                        'dat_noisy','Xn','Yn','Zn'])

        for theta in tqdm(np.linspace(0,np.pi/2,num = N)): 
            for n in rounds:
                Xc = scm_experiment(theta,n,0,measure='x')
                Yc = scm_experiment(theta,n,0,measure='y')
                Zc = scm_experiment(theta,n,0,measure='z')
                dat_clean = np.arctan2(Xc,Zc)/2
                Xn = scm_experiment(theta,n,1,measure='x')
                Yn = scm_experiment(theta,n,1,measure='y')
                Zn = scm_experiment(theta,n,1,measure='z')
                dat_noisy = np.arctan2(Xn,Zn)/2
                writer.writerow([theta,n,dat_clean,Xc,Yc,Zc,dat_noisy,Xn,Yn,Zn])

rounds = [1,2,3,4,5,6,7,8,9]
N = 25

if __name__ == '__main__':
    acquiring_data()

100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [2:03:24<00:00, 296.19s/it]
